<a href="https://colab.research.google.com/github/seanlin2000/cs182_project/blob/main/Sean_Model_Environment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!git clone https://github.com/seanlin2000/cs182_project.git


/content/cs182_project


In [ ]:
!./get_data.sh

Streaming output truncated to the last 5000 lines.
  inflating: tiny-imagenet-200/train/n02231487/images/n02231487_262.JPEG  
  inflating: tiny-imagenet-200/train/n02231487/images/n02231487_108.JPEG  
  inflating: tiny-imagenet-200/train/n02231487/images/n02231487_18.JPEG  
  inflating: tiny-imagenet-200/train/n02231487/images/n02231487_211.JPEG  
  inflating: tiny-imagenet-200/train/n02231487/images/n02231487_134.JPEG  
  inflating: tiny-imagenet-200/train/n02231487/images/n02231487_365.JPEG  
  inflating: tiny-imagenet-200/train/n02231487/images/n02231487_360.JPEG  
  inflating: tiny-imagenet-200/train/n02231487/images/n02231487_185.JPEG  
  inflating: tiny-imagenet-200/train/n02231487/images/n02231487_178.JPEG  
 extracting: tiny-imagenet-200/train/n02231487/images/n02231487_263.JPEG  
  inflating: tiny-imagenet-200/train/n02231487/images/n02231487_32.JPEG  
  inflating: tiny-imagenet-200/train/n02231487/images/n02231487_131.JPEG  
  inflating: tiny-imagenet-200/train/n02231487/imag

In [8]:
from models.convengers import *
from models.solver import *

In [9]:
from __future__ import print_function, division

# import sys
# sys.path.append(root_folder)
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.nn.functional as F
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import pathlib
# from utils import dictionary, preprocess_val

In [6]:
dataset_folder = "./data/tiny-imagenet-200/"
data_dir = pathlib.Path(dataset_folder)

# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(256),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.RandomApply([transforms.GaussianBlur(kernel_size=5)], p=0.25),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.RandomApply([transforms.GaussianBlur(kernel_size=5)], p=0.25),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Load the data
image_datasets = {x: datasets.ImageFolder(data_dir / x, data_transforms[x]) for x in ['train', 'val']}
# Set num_workers=2 when we use CPU, 4 when we use GPU, batch size needs to be smaller for weaker CPUs
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=10, shuffle=True, num_workers=2,pin_memory=False) for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [18]:
models = []
resnet_names = ['resnet-norman.pt', 'resnet-sean1.pt', 'resnet-sean2.pt']
inception_names = ['inception-norman.pt', 'inception-sean1.pt', 'inception-sean2.pt']
for name in resnet_names:
  ckpt = torch.load(name)
  model = Thor(num_blocks=1, requires_grad=True)
  model.load_state_dict(ckpt['overnight'])
  model = model.to(device)
  model.eval()
  models.append(model)

for name in inception_names:
  ckpt = torch.load(name)
  model = IronMan(num_blocks=1, requires_grad=True)
  model.load_state_dict(ckpt['overnight'])
  model = model.to(device)
  model.eval()
  models.append(model)

Downloading: "https://download.pytorch.org/models/resnet101-5d3b4d8f.pth" to /root/.cache/torch/hub/checkpoints/resnet101-5d3b4d8f.pth


Downloading: "https://download.pytorch.org/models/inception_v3_google-1a9a5a14.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-1a9a5a14.pth


In [14]:
!git pull origin main

remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 4 (delta 2), reused 4 (delta 2), pack-reused 0
Unpacking objects: 100% (4/4), done.
From https://github.com/seanlin2000/cs182_project
 * branch            main       -> FETCH_HEAD
   58a9502..3a728de  main       -> origin/main
Updating 58a9502..3a728de
error: Your local changes to the following files would be overwritten by merge:
	models/convengers.py
Please commit your changes or stash them before you merge.
Aborting


In [15]:
ensemble_model = Ensemble()

TypeError: ignored

In [61]:
model_solver = NickFury("ensemble", model, dataloaders, dataset_sizes, device)

In [70]:
model_solver.accuracy("val")

KeyboardInterrupt: ignored

In [ ]:
from advertorch.attacks import PGDAttack
from advertorch.attacks import GradientSignAttack
from advertorch.utils import predict_from_logits
from advertorch_examples.utils import get_mnist_test_loader
from advertorch_examples.utils import _imshow
import matplotlib.pyplot as plt
from advertorch.defenses import MedianSmoothing2D
from advertorch.defenses import BitSqueezing
from advertorch.defenses import JPEGFilter

In [ ]:
adversary = GradientSignAttack(model, loss_fn=nn.CrossEntropyLoss(), eps=0.15)
bits_squeezing = BitSqueezing(bit_depth=5)
median_filter = MedianSmoothing2D(kernel_size=3)
jpeg_filter = JPEGFilter(10)

defense = nn.Sequential(
    bits_squeezing,
    median_filter,
)

defense = defense.to(device)
for idx, (images, labels) in enumerate(dataloaders['train']):
  images = images.to(device)
  labels = labels.to(device)
  adv_images = adversary.perturb(images, labels)
  def_images = defense(images)
  def_adv_images = defense(adv_images)
  def_images = def_images.to(device)
  def_adv_images = def_adv_images.to(device)
  pred_cln = predict_from_logits(model(images))
  pred_adv = predict_from_logits(model(adv_images))
  pred_def = predict_from_logits(model(def_images))
  pred_def_adv = predict_from_logits(model(def_adv_images))

  print("Clean preds were:", pred_cln)
  correct = sum([labels[i] == pred_cln[i] for i in range(len(labels))]).item()
  print("Accuracy is:", correct / len(labels))

  print("Adversarial preds were:", pred_adv)
  adv_correct = sum([labels[i] == pred_adv[i] for i in range(len(labels))]).item()
  print("Adversarial accuracy is", adv_correct / len(labels))

  print("Clean defended preds were", pred_def)
  def_correct = sum([labels[i] == pred_def[i] for i in range(len(labels))]).item()
  print("Clean defense accuracy is", def_correct / len(labels))

  print("Adversarial defended preds were", pred_def_adv)
  def_adv_correct = sum([labels[i] == pred_def_adv[i] for i in range(len(labels))]).item()
  print("Adversarial defense accuracy is", def_adv_correct / len(labels))

  print("Labels were:", labels)
  break

plt.tight_layout()
plt.show()

Clean preds were: tensor([101, 140,  13,  77,  85, 116,  57,  91,  96, 170], device='cuda:0')
Accuracy is: 0.6
Adversarial preds were: tensor([163,  89,  13,  77, 152,  89, 152, 152, 159,  14], device='cuda:0')
Adversarial accuracy is 0.2
Clean defended preds were tensor([101, 140,  13,  77,  85, 116,  57,  91, 152, 170], device='cuda:0')
Clean defense accuracy is 0.7
Adversarial defended preds were tensor([134,  63,   4,  77,  88,  89, 152, 134, 159,  14], device='cuda:0')
Adversarial defense accuracy is 0.1
Labels were: tensor([132, 140,  13,  77, 134, 116,  57, 151, 152, 170], device='cuda:0')


<Figure size 432x288 with 0 Axes>